<h3>Imports</h3>

In [ ]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms.functional as tr
import torchvision.transforms.v2.functional as trv2
from torchvision.transforms import RandomRotation
from torchvision.transforms.functional import pil_to_tensor
import pandas as pd
import torch.nn.functional as F
from torchvision.transforms import InterpolationMode
import matplotlib.pyplot as plt
import math
import time
import os

<h3>Global Variables</h3>

In [ ]:
input_file_dir = "/home/tyler/Downloads/NumaGuard-main/data/videos/clean_2.csv"
src_dir = "/home/tyler/Downloads/NumaGuard-main/data/videos_2/"

In [ ]:
best_model_path = "./best_model/best-parameters.pt"
last_epoch_model_path = "./best_model/last-epoch-correction-parameters.pt"
best_loss_path = "./best_model/best-model-loss.txt"
os.makedirs("./best_model/", exist_ok=True)

In [ ]:
num_of_inputs = 5
num_of_outputs = 4
num_of_frames = 30

hidden_size = 256
num_of_rnn_layers = 1
embedding_size = 3072

In [ ]:
batch_size = 2

In [ ]:
torch.manual_seed(12)

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
print(f"Using {device} device")

In [ ]:
TRAIN_LOSS_KEY = "Training Loss"
TRAIN_ACCURACY_KEY = "Training Accuracy"

VAL_LOSS_KEY = "Validation Loss"
VAL_ACCURACY_KEY = "Validation Accuracy"

TEST_LOSS_KEY = "Testing Loss"
TEST_ACCURACY_KEY = "Testing Accuracy"

<h3>Aux Functions</h3>

In [ ]:
def get_best_loss(current_loss: float, model):
    
    try:
        file = open(best_loss_path, "r+")
    except:
        file = open(best_loss_path, "w+")
        file.write("1")
        file.close()
        get_best_loss(current_loss, model)
        return
        
    line = file.readline()
    best_loss = float(line.replace("\n", ""))

    got_new_loss = False

    if current_loss < best_loss:
        file.seek(0)
        file.write(str(current_loss))
        file.truncate()

        torch.save(model.state_dict(), best_model_path)
        print(f"New best loss!")
        got_new_loss = True

    file.close()

    return got_new_loss

In [ ]:
def get_line(list: list):
    line = [str(x) for x in list]
    line = ','.join(line)
    line += "\n"
    line = line.replace("]", "").replace("[", "").replace(" ", "")

    return line

In [ ]:
def get_dataloader(dataset):
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - (train_size + val_size) 
    train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader, val_dataloader

In [ ]:
def show_and_save_graph(metrics:dict, ylabel:str,  key_one:str, key_two:str=None):
    
    metric_one = metrics[key_one]
    metric_two = metrics[key_two]

    plt.plot([i + 1 for i in range(len(metric_one))] ,metric_one)
    plt.plot([i + 1 for i in range(len(metric_two))] ,metric_two)

    if key_two is not None:

        title = f'{key_one} and {key_two} vs Epoch'

        plt.xlabel('Epoch')
        plt.ylabel(ylabel)
        plt.title(title)
        plt.legend([key_one, key_two])

    else:
        title = f'{key_one} vs Epoch'

        plt.xlabel('Epoch')
        plt.ylabel(ylabel)
        plt.title(title)
        plt.legend([key_one])
    
    plt.grid()

    os.makedirs("./Figures/", exist_ok=True) 
    plt.savefig(f"./Figures/{title}_{int(time.time())}")

    plt.show()

In [ ]:
def init_metrics(*args):

    metrics = {}

    for key in args:
        metrics[key] = []
    
    return metrics

<h3>Custom Dataset Init</h3>

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self):
        df = pd.read_csv(input_file_dir, header=0, dtype=str)
        self.df = df
        self.predictors = df["file_name"].to_numpy()
        self.pins = df["pin"].to_numpy()
        

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if idx >= self.__len__():
            raise StopIteration
        
        file_name = self.predictors[idx]
        predictors = torch.load(f"{src_dir}{file_name}")
        masks = torch.load(f"{src_dir}{file_name}_mask")

        pin = str(self.pins[idx])

        target = [pin[0], pin[1], pin[2], pin[3]]
        target = [int(x) for x in target]

        target = torch.tensor(target).float() / 9

        sample = (predictors, masks, target)

        return sample

In [ ]:
custom_dataset = CustomDataSet()
train_dataloader, test_dataloader, val_dataloader = get_dataloader(custom_dataset)

In [ ]:
for i, (sample, mask, output) in enumerate(custom_dataset):
    # print(f'i : {i} input_file: {sample}{sample.shape}')
    print(f'output_file: {output}{output.shape}')
    # print(f'mask: {mask}{mask.shape}')
    break

<h3>Neural Net Classes</h3>

In [ ]:
class SkipConnection(nn.Module):
    def __init__(self, in_channels, out_channels):

        super().__init__()

        self.conv2d = nn.Conv2d(in_channels, out_channels, 1, padding=0)
    
    def forward(self, x):
        x = self.conv2d(x)
        
        return x

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_features, out_features, 3, padding=1),
            nn.ELU()
        )
        self.cnn2 = nn.Sequential(
                nn.Conv2d(out_features, out_features, 3, padding=1),
                nn.ELU()
        )

        self.skip1 = SkipConnection(in_features, out_features)
        self.pool = nn.MaxPool2d((2,2))
        

    
    def forward(self, x1):

        x2 = self.cnn1(x1)
        x2 = self.cnn2(x2)

        x1 = self.skip1(x1)

        x2 += x1

        x2 = F.elu(x2)
        x2 = self.pool(x2)

        return x2

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

        base = 32

        self.start = nn.Sequential(
            nn.Conv2d(3, base, 7, padding=3),
            nn.ELU(),
            nn.MaxPool2d((2,2))
        )
        
        self.block1 = ConvBlock(base, base*2)
        # self.block2 = ConvBlock(base*2, base*3)
        # self.block3 = ConvBlock(base*3, base*3)
        # self.block4 = ConvBlock(base*3, base*3)
        self.last = nn.Conv2d(base*2, 1, 1, padding=0)

        self.flatten = nn.Flatten()
        
    def forward(self, x1):
        
        x1 = self.start(x1)
        x1 = self.block1(x1) 
        # x1 = self.block2(x1) 
        # x1 = self.block3(x1) 
        # x1 = self.block4(x1) 
        
        x1 = self.last(x1)
        x1 = F.elu(x1)
        x1 = self.flatten(x1)

        return x1

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.gru = nn.GRU(embedding_size, hidden_size, num_of_rnn_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, num_of_outputs)
        
    def forward(self, x1, hn):

        x1, hn = self.gru(x1, hn)
        hn = hn[0]

        x1 = self.fc1(hn)
        x1 = F.elu(x1)
        x1 = self.fc2(x1)

        return x1, hn

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = Encoder()
        self.decoder = Decoder()
        
    def forward(self, x1, masks):

        holder = torch.zeros((x1.shape[0], num_of_outputs), device=device)

        for i, batch in enumerate(x1):
            
            hn = None
            yhat = None

            for k, frame in enumerate(batch):

                mask = masks[i, k]

                if mask == 0:
                    break

                if k == 0:
                    hn = torch.zeros(num_of_rnn_layers, hidden_size, requires_grad=True, device=device)

                yhat = self.encoder(frame)
                yhat, hn = self.decoder(yhat, hn)
                # pred = torch.argmax(yhat, dim=0)
                hn = hn.unsqueeze(0)
                
            holder[i] = yhat
                
        return holder

<h3>Model Init</h3>

In [ ]:
model = NeuralNet().to(device)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Params: {pytorch_total_params:,}")

In [ ]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(list(model.parameters()), lr=0.00001)
metrics = init_metrics(TRAIN_LOSS_KEY, TRAIN_ACCURACY_KEY, VAL_LOSS_KEY, VAL_ACCURACY_KEY)

<h3>Train and Test Init</h3>

In [ ]:
def train(dataloader, optimizer, loss_fn):
    model.train()
    size = 0

    loss, correct = 0, 0
    correct_makes, correct_misses = 0, 0
    incorrect_makes, incorrect_misses = 0, 0

    loss_history = metrics[TRAIN_LOSS_KEY]
    accuracy_history = metrics[TRAIN_ACCURACY_KEY]
    
    for X, mask, y in dataloader:
        X, mask, y = X.to(device), mask.to(device), y.to(device)

        pred = model(X, mask)

        loss = loss_fn(pred, y)

        # pred = torch.argmax(pred, dim=1)
        # y = torch.argmax(y, dim=1)

        # for k in range(len(pred)):
        #     if pred[k] == 0 and y[k] == 0:
        #         correct_makes += 1
        #     elif pred[k] == 1 and y[k] == 1:
        #         correct_misses += 1
        #     elif pred[k] == 0 and y[k] != 0:
        #         incorrect_makes += 1
        #     elif pred[k] == 1 and y[k] != 1:
        #         incorrect_misses += 1

        # corrects = torch.eq(pred,y).int()
        
        # correct += corrects.sum().item()

        # size += corrects.numel()

        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

    loss_history.append(loss.item())

    if incorrect_makes == 0:
        incorrect_makes = 1
    
    if incorrect_misses == 0:
        incorrect_misses = 1
    
    # correct /= size
    # precision = correct_makes / (correct_makes + incorrect_makes)
    # recall = correct_makes / (correct_makes + incorrect_misses)
    # accuracy = 100*correct
    # accuracy_history.append(accuracy)
    # print(f"Train Accuracy: {(accuracy):>0.3f}%\t Train Loss: {loss:>12f}")
    print(f"Train Loss: {loss:>12f}")
    # print(f"Train Precision: {precision}\t Train Recall: {recall}")

In [ ]:
def val(dataloader, loss_fn):
    size = 0

    num_batches = len(dataloader)

    model.eval()
    
    loss, correct = 0, 0
    correct_makes, correct_misses = 0, 0
    incorrect_makes, incorrect_misses = 0, 0

    loss_history = metrics.get(VAL_LOSS_KEY, [])
    accuracy_history = metrics.get(VAL_ACCURACY_KEY, [])
    
    with torch.no_grad():
        for X, mask, y in dataloader:
            X, mask, y = X.to(device), mask.to(device), y.to(device)

            pred = model(X, mask)

            loss += loss_fn(pred, y)

            # pred = torch.argmax(pred, dim=1)
            # y = torch.argmax(y, dim=1)

            # for k in range(len(pred)):
            #     if pred[k] == 0 and y[k] == 0:
            #         correct_makes += 1
            #     elif pred[k] == 1 and y[k] == 1:
            #         correct_misses += 1
            #     elif pred[k] == 0 and y[k] != 0:
            #         incorrect_makes += 1
            #     elif pred[k] == 1 and y[k] != 1:
            #         incorrect_misses += 1

            # corrects = torch.eq(pred,y).int()
            
            # correct += corrects.sum().item()

            # size += corrects.numel()

    loss /= num_batches
    loss_history.append(loss.item())

    if incorrect_makes == 0:
        incorrect_makes = 1
    
    if incorrect_misses == 0:
        incorrect_misses = 1

    # correct /= size
    # accuracy = 100*correct
    # precision = correct_makes / (correct_makes + incorrect_makes)
    # recall = correct_makes / (correct_makes + incorrect_misses)
    # accuracy_history.append(accuracy)
    # print(f"Val Accuracy: {(accuracy):>0.3f}%\t Val Loss: {loss:>12f}")
    print(f"Val Loss: {loss:>12f}")
    # print(f"Val Precision: {precision}\t Val Recall: {recall}")

In [ ]:
def test(dataloader, loss_fn):
    size = 0

    encoder.eval()
    decoder.eval()

    num_batches = len(dataloader)
    
    loss, correct = 0, 0

    loss_history = metrics.get(TEST_LOSS_KEY, [])
    accuracy_history = metrics.get(TEST_ACCURACY_KEY, [])
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = encoder(X)

            pred = pred.clamp(0, 1)
            pred = torch.nan_to_num(pred)
            
            loss += loss_fn(pred, y)

            pred = torch.argmax(pred, dim=1)
            y = torch.argmax(y, dim=1)

            corrects = torch.eq(pred,y).int()

            correct += corrects.sum().item()

            size += corrects.numel()

    loss /= num_batches
    loss_history.append(loss.item())

    correct /= size
    accuracy = 100*correct
    accuracy_history.append(accuracy)
    print(f"Test Accuracy: {(accuracy):>0.3f}%\tTest Loss: {loss:>12f}")

In [ ]:
def print_metrics(start, t, t_with_best_loss, best_loss):
    train_loss_hist = metrics[TRAIN_LOSS_KEY]
    val_loss_hist = metrics[VAL_LOSS_KEY]

    if t > 1:
        train_loss_dif = (train_loss_hist[-2] - train_loss_hist[-1]) * 100
        val_loss_dif = (val_loss_hist[-2] - val_loss_hist[-1]) * 100
        train_val_loss_dif = (train_loss_hist[-1] - val_loss_hist[-1]) * 100
        print()
        print(f"Train Loss Difference: {train_loss_dif:>0.4f}\t\tVal Loss Difference: {val_loss_dif:>0.4f}")
        print(f"Train Val Loss Difference: {train_val_loss_dif:>0.4f}")
        print()

    got_new_loss = get_best_loss(val_loss_hist[-1], model)

    if got_new_loss:
        t_with_best_loss = t
        best_loss = val_loss_hist[-1]
    
    t_since_best_loss = t - t_with_best_loss
    
    print(f"Epoch with best loss: {t_with_best_loss}\t\tBest Loss: {best_loss:12f}")
    print(f"Epochs Since Best Loss: {t_since_best_loss}")

    print(f"Run Time: {round((time.time() - start), 2)}s")

    print()

    return t_with_best_loss, best_loss

<h3>Training and Validation</h3>

In [ ]:
t = 0
t_with_best_loss = 0
best_loss = 0

while True:
    start = time.time()
    t = t + 1

    print(f"Epoch {t}\n-------------------------------")

    train(train_dataloader, optimizer, loss)
    val(val_dataloader, loss)

    t_with_best_loss, best_loss = print_metrics(start, t, t_with_best_loss, best_loss)

In [ ]:
makes = 0
misses = 0
for sample, target in train_dataloader:
    for batch in target:
        if batch[0] == 0:
            misses += 1
        else:
            makes += 1

print(f"Makes: {makes} Misses: {misses}")

In [ ]:
show_and_save_graph(metrics, "Loss", TRAIN_LOSS_KEY, VAL_LOSS_KEY)

In [ ]:
test1, y = custom_dataset[0]
test1 = test1.unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(test1)

In [ ]:
pred = pred.cpu().squeeze()
y = y.squeeze()
for i, val in enumerate(pred):
    print(f"pred: {val} y: {y[i]} dif: {(val - y[i])}")

In [ ]:
plt.plot([i + 1 for i in range(len(pred))] ,pred)
plt.plot([i + 1 for i in range(len(y))] ,y)
plt.legend(["Pred", "Y"])
plt.grid()
plt.show()